In [7]:
# import linear algebra and data manipulation libraries
import numpy as np
import pandas as pd

# import matplotlib for plotting
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

# import helper libraries
import requests
from io import BytesIO # Use When expecting bytes-like objects
import pickle
from collections import OrderedDict
import os
from os import path
import time
import argparse

# import PIL for image manipulation
from PIL import Image

# import machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# import pytorch
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import sys
sys.path.append("src/")
import image_utils
from image_utils import add_flipped_and_rotated_images
from simple_conv_nn import SimpleCNN

In [8]:
def load_data():
    """
    Function loads quick draw dataset. If no data is loaded yet, the datasets
    are loaded from the web. If there are already loaded datasets, then data
    is loaded from the disk (pickle files).

    INPUTS: None

    OUTPUT:
        X_train - train dataset
        y_train - train dataset labels
        X_test - test dataset
        y_test - test dataset labels
    """
    print("Loading data \n")

    # Check for already loaded datasets
    if not(path.exists('xtrain_doodle.pickle')):
        # Load from web
        print("Loading data from the web \n")

        # Classes we will load
        categories = ['bee', 'cat', 'cow', 'dog', 'duck', 'horse', 'pig', 'rabbit', 'snake', 'whale']

        # Dictionary for URL and class labels
        URL_DATA = {}
        for category in categories:
            URL_DATA[category] = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/' + category +'.npy'

        # Load data for classes in dictionary
        classes_dict = {}
        for key, value in URL_DATA.items():
            response = requests.get(value)
            classes_dict[key] = np.load(BytesIO(response.content))

        # Generate labels and add labels to loaded data
        for i, (key, value) in enumerate(classes_dict.items()):
            value = value.astype('float32')/255.
            if i == 0:
                classes_dict[key] = np.c_[value, np.zeros(len(value))]
            else:
                classes_dict[key] = np.c_[value,i*np.ones(len(value))]

        # Create a dict with label codes
        label_dict = {0:'bee', 1:'cat', 2:'cow', 3:'dog', 4:'duck',
                      5:'horse', 6:'pig', 7:'rabbit', 8:'snake', 9:'whale'}

        lst = []
        for key, value in classes_dict.items():
            lst.append(value[:3000])
        doodles = np.concatenate(lst)

        # Split the data into features and class labels (X & y respectively)
        y = doodles[:,-1].astype('float32')
        X = doodles[:,:784]

        # Split each dataset into train/test splits
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)
    else:
        # Load data from pickle files
        print("Loading data from pickle files \n")

        file = open("xtrain_doodle.pickle",'rb')
        X_train = pickle.load(file)
        file.close()

        file = open("xtest_doodle.pickle",'rb')
        X_test = pickle.load(file)
        file.close()

        file = open("ytrain_doodle.pickle",'rb')
        y_train = pickle.load(file)
        file.close()

        file = open("ytest_doodle.pickle",'rb')
        y_test = pickle.load(file)
        file.close()

    return X_train, y_train, X_test, y_test, classes_dict

In [9]:
categories = ['bee', 'cat', 'cow', 'dog', 'duck', 'horse', 'pig', 'rabbit', 'snake', 'whale']

URL_DATA = {}
for category in categories:
    URL_DATA[category] = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/' + category +'.npy'

URL_DATA

{'bee': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bee.npy',
 'cat': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy',
 'cow': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cow.npy',
 'dog': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dog.npy',
 'duck': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/duck.npy',
 'horse': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/horse.npy',
 'pig': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pig.npy',
 'rabbit': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/rabbit.npy',
 'snake': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/snake.npy',
 'whale': 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/whale.npy'}

In [10]:
classes_dict = {}
for key, value in URL_DATA.items():
    response = requests.get(value)
    classes_dict[key] = np.load(BytesIO(response.content))

In [11]:
classes_dict

{'bee': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'cat': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'cow': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'dog': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'duck': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0

In [12]:
len(classes_dict['dog'])

152159

In [13]:
len(classes_dict['cat'])

123202

In [14]:
# Generate labels and add labels to loaded data
for i, (key, value) in enumerate(classes_dict.items()):
    value = value.astype('float32')/255.
    if i == 0:
        classes_dict[key] = np.c_[value, np.zeros(len(value))]
    else:
        classes_dict[key] = np.c_[value,i*np.ones(len(value))]

In [15]:
classes_dict['bee']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
classes_dict['bee'][0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.21568628, 0.33725491,
       0.40000001, 0.40000001, 0.21960784, 0.        , 0.     

In [17]:
label_dict = {0:'bee', 1:'cat', 2:'cow', 3:'dog', 4:'duck',
              5:'horse', 6:'pig', 7:'rabbit', 8:'snake', 9:'whale'}

lst = []
for key, value in classes_dict.items():
    lst.append(value[:3000])
doodles = np.concatenate(lst)

In [18]:
len(doodles)

30000

In [19]:
doodles[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.21568628, 0.33725491,
       0.40000001, 0.40000001, 0.21960784, 0.        , 0.     

In [20]:
y = doodles[:,-1].astype('float32')
X = doodles[:,:784]

In [21]:
y

array([0., 0., 0., ..., 9., 9., 9.], dtype=float32)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
len(X_train)

21000

In [25]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [26]:
len(X_train[0])

784

In [27]:
X_train[0][0]

0.0

In [28]:
X_train[0][1]

0.0

In [30]:
len(y_train)

21000

In [46]:
def save_data(X_train, y_train, X_test, y_test, force = False):
    
    # Check for already saved files
    if not(path.exists('data/xtrain.pickle')) or force:
        # Save X_train dataset as a pickle file
        with open('data/xtrain.pickle', 'wb') as f:
            pickle.dump(X_train, f)

        # Save X_test dataset as a pickle file
        with open('data/xtest.pickle', 'wb') as f:
            pickle.dump(X_test, f)

        # Save y_train dataset as a pickle file
        with open('data/ytrain.pickle', 'wb') as f:
            pickle.dump(y_train, f)

        # Save y_test dataset as a pickle file
        with open('data/ytest.pickle', 'wb') as f:
            pickle.dump(y_test, f)

In [47]:
save_data(X_train, y_train, X_test, y_test)

Saving data 



In [48]:
type(X_train)

numpy.ndarray

In [91]:
type(X_test)

numpy.ndarray

In [92]:
type(y_train)

numpy.ndarray

In [49]:
len(X_train)

21000

In [28]:
len(X_test)

9000

In [32]:
def build_model(input_size, output_size, architecture = 'nn', dropout = 0.0):
    if (architecture == 'nn'):
        # Build a feed-forward network
        model = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(input_size, 128)),
                              ('relu1', nn.ReLU()),
                              ('fc2', nn.Linear(128, 100)),
                              ('bn2', nn.BatchNorm1d(num_features=100)),
                              ('relu2', nn.ReLU()),
                              ('dropout', nn.Dropout(dropout)),
                              ('fc3', nn.Linear(100, 64)),
                              ('bn3', nn.BatchNorm1d(num_features=64)),
                              ('relu3', nn.ReLU()),
                              ('logits', nn.Linear(64, output_size))]))
    else:
        if (architecture == 'conv'):
            # Build a simple convolutional network
            model = SimpleCNN(64, 10)
    return model

In [33]:
rnn_model = build_model(input_size=784, output_size=10)

In [34]:
rnn_model

Sequential(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (dropout): Dropout(p=0.0, inplace=False)
  (fc3): Linear(in_features=100, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (logits): Linear(in_features=64, out_features=10, bias=True)
)

In [35]:
cnn_model = build_model(input_size = 784, output_size=10, architecture='conv')

In [36]:
cnn_model

SimpleCNN(
  (conv1): Conv2d(1, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3528, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [38]:
X_train = pickle.load(open('data/xtrain.pickle', 'rb'))
X_test = pickle.load(open('data/xtest.pickle', 'rb'))
y_train = pickle.load(open('data/ytrain.pickle', 'rb'))
y_test = pickle.load(open('data/ytest.pickle', 'rb'))

In [39]:
def shuffle(X_train, y_train):

    from sklearn.utils import shuffle
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=42)

    y_train_shuffled = y_train_shuffled.reshape((X_train.shape[0], 1))

    X_train_shuffled = torch.from_numpy(X_train_shuffled).float()
    y_train_shuffled = torch.from_numpy(y_train_shuffled).long()

    return X_train_shuffled, y_train_shuffled

In [40]:
X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)

In [41]:
type(X_train_shuffled)

torch.Tensor

In [42]:
type(y_train_shuffled)

torch.Tensor

In [43]:
def fit_model(model, X_train, y_train, epochs = 100, n_chunks = 1000, learning_rate = 0.003, weight_decay = 0, optimizer = 'SGD'):

    print("Fitting model with epochs = {epochs}, learning rate = {lr}\n"\
    .format(epochs = epochs, lr = learning_rate))
    criterion = nn.CrossEntropyLoss()
    if (optimizer == 'SGD'):
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay= weight_decay)
    else:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay= weight_decay)
    print_every = 100
    steps = 0
    for e in range(epochs):
        running_loss = 0
        images = torch.chunk(X_train, n_chunks)
        labels = torch.chunk(y_train, n_chunks)
        for i in range(n_chunks):
            steps += 1
            optimizer.zero_grad()

            # Forward and backward passes
            output = model.forward(images[i])
            loss = criterion(output, labels[i].squeeze())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every))
                running_loss = 0


In [44]:
fit_model(rnn_model, X_train_shuffled, y_train_shuffled)

Fitting model with epochs = 100, learning rate = 0.003

Epoch: 1/100...  Loss: 2.2037
Epoch: 1/100...  Loss: 2.0504
Epoch: 1/100...  Loss: 1.9417
Epoch: 1/100...  Loss: 1.8689
Epoch: 1/100...  Loss: 1.8064
Epoch: 1/100...  Loss: 1.7327
Epoch: 1/100...  Loss: 1.6904
Epoch: 1/100...  Loss: 1.7019
Epoch: 1/100...  Loss: 1.6900
Epoch: 1/100...  Loss: 1.6214
Epoch: 2/100...  Loss: 1.5917
Epoch: 2/100...  Loss: 1.5950
Epoch: 2/100...  Loss: 1.5655
Epoch: 2/100...  Loss: 1.5290
Epoch: 2/100...  Loss: 1.5380
Epoch: 2/100...  Loss: 1.4726
Epoch: 2/100...  Loss: 1.4581
Epoch: 2/100...  Loss: 1.5071
Epoch: 2/100...  Loss: 1.5234
Epoch: 2/100...  Loss: 1.4467
Epoch: 3/100...  Loss: 1.4421
Epoch: 3/100...  Loss: 1.4452
Epoch: 3/100...  Loss: 1.4261
Epoch: 3/100...  Loss: 1.3853
Epoch: 3/100...  Loss: 1.4136
Epoch: 3/100...  Loss: 1.3427
Epoch: 3/100...  Loss: 1.3311
Epoch: 3/100...  Loss: 1.3868
Epoch: 3/100...  Loss: 1.4151
Epoch: 3/100...  Loss: 1.3340
Epoch: 4/100...  Loss: 1.3405
Epoch: 4/100..

In [45]:
def load_model(architecture = 'nn', filepath = 'checkpoint.pth'):
    """
    Function loads the model from checkpoint.

    INPUT:
        architecture - model architecture ('nn' - for fully connected neural network, 'conv' - for convolutional neural
        network)
        filepath - path for the saved model

    OUTPUT:
        model - loaded pytorch model
    """

    print("Loading model from {} \n".format(filepath))

    if architecture == 'nn':
        checkpoint = torch.load(filepath)
        input_size = checkpoint['input_size']
        output_size = checkpoint['output_size']
        hidden_sizes = checkpoint['hidden_layers']
        dropout = checkpoint['dropout']
        model = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                              ('relu1', nn.ReLU()),
                              ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                              ('bn2', nn.BatchNorm1d(num_features=hidden_sizes[1])),
                              ('relu2', nn.ReLU()),
                              ('dropout', nn.Dropout(dropout)),
                              ('fc3', nn.Linear(hidden_sizes[1], hidden_sizes[2])),
                              ('bn3', nn.BatchNorm1d(num_features=hidden_sizes[2])),
                              ('relu3', nn.ReLU()),
                              ('logits', nn.Linear(hidden_sizes[2], output_size))]))
        model.load_state_dict(checkpoint['state_dict'])

    else:
        checkpoint = torch.load(filepath)
        model = SimpleCNN()
        model.load_state_dict(checkpoint['state_dict'])

    return model

In [47]:
model = load_model(architecture = 'nn', filepath = 'data/checkpoint.pth')

Loading model from data/checkpoint.pth 



In [50]:
def get_preds(model, input, architecture = 'nn'):
    """
    Function to get predicted probabilities from the model for each class.

    INPUT:
        model - pytorch model
        input - (tensor) input vector

    OUTPUT:
        ps - (tensor) vector of predictions
    """

    # Turn off gradients to speed up this part
    with torch.no_grad():
        if architecture == 'nn':
            logits = model.forward(input)
        else:
            image = input.numpy()
            image = image.reshape(image.shape[0], 1, 28, 28)
            logits = model.forward(torch.from_numpy(image).float())

    ps = F.softmax(logits, dim=1)
    return ps

In [48]:
def evaluate_model(model, train, y_train, test, y_test, architecture = 'nn'):
    """
    Function to print out train and test accuracy of the model.

    INPUT:
        model - pytorch model
        train - (tensor) train dataset
        y_train - (numpy) labels for train dataset
        test - (tensor) test dataset
        y_test - (numpy) labels for test dataset

    OUTPUT:
        accuracy_train - accuracy on train dataset
        accuracy_test - accuracy on test dataset
    """
    train_pred = get_preds(model, train, architecture)
    train_pred_labels = get_labels(train_pred)

    test_pred = get_preds(model, test, architecture)
    test_pred_labels = get_labels(test_pred)

    accuracy_train = accuracy_score(y_train, train_pred_labels)
    accuracy_test = accuracy_score(y_test, test_pred_labels)

    print("Accuracy score for train set is {} \n".format(accuracy_train))
    print("Accuracy score for test set is {} \n".format(accuracy_test))

    return accuracy_train, accuracy_test

In [52]:
def get_labels(pred):
    """
        Function to get the vector of predicted labels for the images in
        the dataset.

        INPUT:
            pred - (tensor) vector of predictions (probabilities for each class)
        OUTPUT:
            pred_labels - (numpy) array of predicted classes for each vector
    """

    pred_np = pred.numpy()
    pred_values = np.amax(pred_np, axis=1, keepdims=True)
    pred_labels = np.array([np.where(pred_np[i, :] == pred_values[i, :])[0] for i in range(pred_np.shape[0])])
    pred_labels = pred_labels.reshape(len(pred_np), 1)

    return pred_labels

In [ ]:
y_test = y_test.reshape((X_test.shape[0], 1))
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [53]:
evaluate_model(model, X_train_shuffled, y_train_shuffled, X_test, y_test, architecture='nn')

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [ ]:
evaluate_model